In [1]:
import pandas as pd
import tensorflow as tf
import pandas as pd
from tensorflow import keras
# Helper libraries
import numpy as np
from IPython.display import display 
# import matplotlib.pyplot as pl
import scipy
import glob,os
import PIL.Image as Image
import pathlib
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img


In [2]:
TRAIN_PATH = '/Users/i516344/PycharmProjects/pythonProject/fashion/print_style/train/'
VAL_PATH = '/Users/i516344/PycharmProjects/pythonProject/fashion/print_style/val/'

In [3]:
img_height = 256
img_width = 256
batch_size = 32

In [4]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
validation_generator = test_datagen.flow_from_directory(
    VAL_PATH, # same directory as training data
    target_size=(img_height, img_width),
    color_mode="rgb",
    class_mode="categorical",
    shuffle=True,
    batch_size=batch_size) # set as validation data
train_generator = train_datagen.flow_from_directory(
    TRAIN_PATH,
    color_mode="rgb",
    class_mode="categorical",
    shuffle=True,
    target_size=(img_height, img_width),
    batch_size=batch_size) # set as training data

Found 7680 images belonging to 4 classes.
Found 17920 images belonging to 4 classes.


In [6]:
resnet50 = keras.applications.ResNet50(include_top=False, input_shape=(img_height,img_width, 3), weights='imagenet')

94773248/94765736 [==============================] - 8s 0us/step


In [7]:
for layer in resnet50.layers:
    layer.trainable = False

In [8]:
model = keras.models.Sequential()
x = keras.layers.GlobalAveragePooling2D()(resnet50.layers[-1].output)
flat1 = tf.keras.layers.Flatten()(x)
class0 = tf.keras.layers.Dense(256, activation='relu')(flat1)
class1 = tf.keras.layers.Dense(64, activation='relu')(class0)
class2 = tf.keras.layers.Dense(8, activation='relu')(class1)
output = tf.keras.layers.Dense(4, activation='sigmoid')(class2)

model = keras.Model(inputs = resnet50.inputs, outputs = output)
model.summary()
# model = keras.Model(inputs = vgg_model.inputs, outputs = output)
# model.summary()
# model.add(resnet50)
# model.add(keras.layers.AveragePooling2D())
# model.add(keras.layers.Flatten())
# model.add(keras.layers.Dense(64,'relu'))
# model.add(keras.layers.Dense(8,'relu'))
# model.add(keras.layers.Dense(4,'sigmoid'))
# model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [9]:
#optimizer with custom learning rate
opt = keras.optimizers.Adam(learning_rate=0.005)
model.compile(optimizer=opt, loss='categorical_crossentropy',
                metrics=['accuracy'])
modelname='ResNet50_V2'
filepath=modelname+"weights-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
history = model.fit(train_generator,epochs=25,
                    validation_data=validation_generator,
                    callbacks=callbacks_list, verbose=1)

Epoch 1/25
560/560 [==============================] - 1989s 4s/step - loss: 1.3983 - accuracy: 0.2504 - val_loss: 1.3871 - val_accuracy: 0.2500

Epoch 00001: val_accuracy improved from -inf to 0.25000, saving model to ResNet50_V2weights-01-0.25.hdf5
Epoch 2/25
560/560 [==============================] - 2022s 4s/step - loss: 1.3867 - accuracy: 0.2515 - val_loss: 1.3868 - val_accuracy: 0.2500

Epoch 00002: val_accuracy did not improve from 0.25000
Epoch 3/25
132/560 [======>.......................] - ETA: 17:38 - loss: 1.3868 - accuracy: 0.2442